In [1]:
import pandas as pd
from bc_power import utils

In [2]:
# read data
# config
data_path = r"/mnt/c/Users/pmcw9/Delta-E/PICS/Data/SESIT/CODERS/data-pull/supply/"
gen_generic_file = "generation_generic.csv"
gen_file = "generators.csv"
hydro_e_file = "hydro_existing.csv"
hydro_c_file = "hydro_cascade.csv"
gen_wup_data_file = r"/mnt/c/Users/pmcw9/Delta-E/PICS/Data/custom/hydro_gen_wup_features.csv"
res_wup_data_file = r"/mnt/c/Users/pmcw9/Delta-E/PICS/Data/custom/hydro_res_wup_features.csv"

# reads
gen_generic = pd.read_csv(data_path + gen_generic_file)
generators = pd.read_csv(data_path + gen_file)
hydro_e_data = pd.read_csv(data_path + hydro_e_file)
cascade_data = pd.read_csv(data_path + hydro_c_file)
gen_wup_data= pd.read_csv(gen_wup_data_file)
res_wup_data= pd.read_csv(res_wup_data_file)

# write path + file
df_hydro_path = r"/mnt/c/Users/pmcw9/Delta-E/PICS/PyPSA_BC/results/interim/hydro_assets.csv"
df_res_path = r"/mnt/c/Users/pmcw9/Delta-E/PICS/PyPSA_BC/results/interim/hydro_reservoirs.csv"

In [3]:
def add_generators_features(row, cid_dict):
    '''
    This function extracts generator features from the generators.csv table
    from CODERS to dictionary which is merging hydro relevant data to create a
    hydro assets file.
    '''
    # Add error checking later
    cid_dict["asset_id"] = utils.convert_cid_2_aid(row['gen_node_code'], row["connecting_node_code"])
    cid_dict["connecting_node_code"] = row["connecting_node_code"]
    cid_dict["num_of_units"] = row["total_num_of_units"]
    cid_dict["latitude"] = row["latitude"]
    cid_dict["longitude"] = row["longitude"]
    cid_dict["capacity"] = row["install_capacity_in_mw"] # Could potentially used effective instead of installed capacity
    cid_dict["annual_avg_energy"] = row["annual_avg_energy_unit_in_gwh/y"]
    cid_dict["gen_type"] = row["gen_type"].lower()
    cid_dict["start_year"] = row["start_year"]
    cid_dict["closure_year"] = row["closure_year"]


def add_cascade_features(row, cid_dict):
    '''
    This function extracts generator features from the hydro_cascade.csv table
    from CODERS to dictionary which is merging hydro relevant data to create a
    hydro assets file.
    '''
    cid_dict["nominal_head"] = "DEFAULT" # Too sparse
    cid_dict["max_spill"] = float(row["max_spill"]) # m3/s
    cid_dict["max_water_discharge"] = row["max_water_discharge"] # m3/s
    cid_dict["min_water_discharge"] = row["min_water_discharge"] # m3/s
    cid_dict["cascade_group"] = row["cascade_group_name"]
    cid_dict["cascade_order"] = row["number"]
    cid_dict["max_storage"] = row["max_storage"]
    cid_dict["min_storage"] = row["min_storage"]

def add_hydro_existing_features(row, cid_dict):
    '''
    This function extracts generator features from the existing.csv table
    from CODERS to dictionary which is merging hydro relevant data to create a
    hydro assets file.
    '''
    cid_dict["upper_reservoir_id"] = row["upper_storage_name"]
    cid_dict["lower_reservoir_id"] = row["lower_storage_name"]
    cid_dict["max_level"] = row["maximum_level"]
    cid_dict["min_level"] = row["minimum_level"]

def add_gen_generic_features(row, cid_dict):
    '''
    This function extracts generator features from the generation_generic.csv table
    from CODERS to dictionary which is merging hydro relevant data to create a
    hydro assets file.
    UPDATE: Likely, may use try other sources from ramping constraints and etc.. in future. This function will need to be updated
            accordingly to accommodate those changes.
    '''
    cid_dict["ramp_up"] = cid_dict["capacity"] * row["ramp_rate_percent_per_min"] # MW/min
    cid_dict["ramp_down"] = cid_dict["capacity"] * row["ramp_rate_percent_per_min"] # MW/min (symmetric assume..)
    cid_dict["min_up"] = row["min_up_time_hours"] # hrs
    cid_dict["min_down"] = row["min_down_time_hours"] # hrs
    cid_dict["min_gen"] = row["min_plant_load"] * cid_dict["capacity"] # hrs
    cid_dict["spinning_reserve_capability"] = row["spinning_reserve_capability"] # ??? (Needs to be corrected...)
    cid_dict["forced_outage_rate"] = row["forced_outage_rate"]
    cid_dict["planned_outage_rate"] = row["planned_outage_rate"]
    cid_dict["start_up_cost_cold"] = row["cold_start_up_costs_$_per_Mwcap"]
    cid_dict["shutdown_cost"] = row["shutdown_cost"]
    cid_dict["typical_plant_size_MW"] = row["typical_plant_size_MW"]
    cid_dict["capital_cost_USD_per_kW"] = row["capital_cost_USD_per_kW"]
    cid_dict["service_life_years"] = row["service_life_years"]
    cid_dict["fixed_om_cost_USD_per_MWyear"] = row["fixed_om_cost_USD_per_MWyear"]
    cid_dict["variable_om_cost_USD_per_MWh"] = row["variable_om_cost_USD_per_MWh"]
    cid_dict["average_fuel_price_USD_per_MMBtu"] = row["average_fuel_price_USD_per_MMBtu"]
    cid_dict["carbon_emissions_tCO2eq_per_MWh"] = row["carbon_emissions_tCO2eq_per_MWh"]


In [4]:
# Accessing features
# data_dict: holds the features which will be required in the final hydro_assets.csv
data_dict = { # replace later with reading a configuration file with the targetted columns
             "asset_id":"DEFAULT",
             "connecting_node_code":"DEFAULT",
             "num_of_units":"DEFAULT",
             "latitude":"DEFAULT",
             "longitude":"DEFAULT",
             "capacity":"DEFAULT",
             "capacity_factor":"DEFAULT",
             "annual_avg_energy":"DEFAULT",
             "p_min":"DEFAULT",
             "nominal_head":"DEFAULT",
             "max_spill":"DEFAULT",
             "max_water_discharge":"DEFAULT",
             "min_water_discharge":"DEFAULT",
             "gen_type":"DEFAULT",
             "cascade_group":"DEFAULT",
             "cascade_order":"DEFAULT",
             "upper_reservoir_id":"DEFAULT",
             "lower_reservoir_id":"DEFAULT",
             "max_level":"DEFAULT",
             "min_level":"DEFAULT",
             "min_storage":"DEFAULT",
             "max_storage":"DEFAULT",
             "ramp_up":"DEFAULT",
             "ramp_down":"DEFAULT",
             "min_up":"DEFAULT",
             "min_down":"DEFAULT",
             "min_gen":"DEFAULT",
             "spinning_reserve_capability":"DEFAULT",
             "forced_outage_rate":"DEFAULT",
             "planned_outage_rate":"DEFAULT",
             "start_up_cost_cold":"DEFAULT",
             "shutdown_cost":"DEFAULT",
             "typical_plant_size_MW":"DEFAULT",
             "capital_cost_USD_per_kW":"DEFAULT",
             "service_life_years":"DEFAULT",
             "fixed_om_cost_USD_per_MWyear":"DEFAULT",
             "variable_om_cost_USD_per_MWh":"DEFAULT",
             "average_fuel_price_USD_per_MMBtu":"DEFAULT",
             "carbon_emissions_tCO2eq_per_MWh":"DEFAULT",
             "start_year":"DEFAULT",
             "closure_year":"DEFAULT"}

hydro_types = set(["hydro_daily","hydro_run","hydro_monthly","hydro_annual"])
component_dict = {}

In [5]:
# (i) get features from generators.csv
mask = (generators['province'] == "BC") & (generators['gen_type'].apply(lambda x: x.lower() in hydro_types))
generators[mask]
for idx,row in generators[mask].iterrows():
    cid = row["gen_node_code"]
    if cid in component_dict:
        print("ERROR Non-unique: Component ID already added.")
        break
    component_dict[cid] = data_dict.copy()
    add_generators_features(row, component_dict[cid])


In [6]:
# (ii) get features from hydro_cascade.csv
for idx,row in cascade_data.iterrows():
    cid = row["gen_node_code"]
    if cid not in component_dict:
        print("ERROR Non-unique: Component ID already added.")
        break
    add_cascade_features(row, component_dict[cid])

In [7]:
# (iii) get features from hydro existing
mask = (hydro_e_data['Province'] == "BC") 
for idx,row in hydro_e_data[mask].iterrows():
    for cid in component_dict.keys(): # many-to-one
        if cid not in component_dict:
            print("ERROR Non-unique: Component ID already added.")
            break
        if component_dict[cid]["asset_id"] == row["connecting_node_code"]:
            add_hydro_existing_features(row, component_dict[cid])
    
    

In [8]:
# (iv) generic hydro dataset
gen_generic.dropna(how="all",inplace=True)
for cid,features in component_dict.items():
    for idx,row in gen_generic.iterrows():
        if features["gen_type"] == row["generation_type"].lower():
            add_gen_generic_features(row, features)

In [9]:
# (v) Customized imputation of select areas
# unwrap and create csv 
# and Custom fix for CID: BC_USR00_GEN, BC_USR02_GEN
df_hydro_gen = pd.DataFrame.from_dict(component_dict,orient='index')
df_hydro_gen.reset_index(inplace=True)
df_hydro_gen.rename({"index":"component_id"}, inplace=True, axis=1)
cid_2_aid_dict = {"BC_USR00_GEN":"BC_USR_GSS","BC_USR02_GEN":"BC_LMN_GSS"} # custom fix
for cid,aid in cid_2_aid_dict.items():
    ind = df_hydro_gen[df_hydro_gen["component_id"] == cid].index[0]
    df_hydro_gen.at[ind,"asset_id"] = aid

In [11]:
# (vi) 
# a) update hydro technical parameters based on WUP generation data
# b) crate csv of hydro generation assets
for ind,row in gen_wup_data.iterrows():
    temp_mask = df_hydro_gen["asset_id"] == row["asset_id"]
    for ser_ind,ser_val in row[row.notnull()].iteritems():
        df_hydro_gen.loc[temp_mask,ser_ind] = ser_val # Use non-empty values from the generation WUP extacted data
 
df_hydro_gen.to_csv(df_hydro_path, index=False)

# (vii) 
# a) create csv of hydro reservoirs
res_wup_data.to_csv(df_res_path, index=False)